## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,9,Polewali,ID,85.75,scattered clouds,-3.4324,119.3435,Lilianto Hotel
1,10,Vaini,TO,78.96,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
2,13,Bambous Virieux,MU,75.22,broken clouds,-20.3428,57.7575,Casa Tia Villa
3,14,Georgetown,MY,80.89,few clouds,5.4112,100.3354,Cititel Penang
4,18,Lethem,GY,77.85,heavy intensity rain,3.3803,-59.7968,Savannah Inn


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,9,Polewali,ID,85.75,scattered clouds,-3.4324,119.3435,Lilianto Hotel
1,10,Vaini,TO,78.96,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
2,13,Bambous Virieux,MU,75.22,broken clouds,-20.3428,57.7575,Casa Tia Villa
3,14,Georgetown,MY,80.89,few clouds,5.4112,100.3354,Cititel Penang
4,18,Lethem,GY,77.85,heavy intensity rain,3.3803,-59.7968,Savannah Inn
5,30,Atuona,PF,76.26,light rain,-9.8000,-139.0333,Villa Enata
6,33,Road Town,VG,82.49,overcast clouds,18.4167,-64.6167,"ZINGARA 76ft Catamaran, full crew-all included"
7,35,Puerto Escondido,MX,76.17,overcast clouds,15.8500,-97.0667,Aldea del Bazar Hotel and Spa
8,42,Hithadhoo,MV,82.02,scattered clouds,-0.6000,73.0833,Scoop Guest House
9,43,Airai,TL,81.54,scattered clouds,-8.9266,125.4092,NaN


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID         0
City            0
Country         0
Max Temp        0
Description     0
Lat             0
Lng             0
Hotel Name     18
dtype: int64

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Polewali,ID,85.75,scattered clouds,-3.4324,119.3435,
1,Vaini,TO,78.96,scattered clouds,-21.2000,-175.2000,
2,Bambous Virieux,MU,75.22,broken clouds,-20.3428,57.7575,
3,Georgetown,MY,80.89,few clouds,5.4112,100.3354,
4,Lethem,GY,77.85,heavy intensity rain,3.3803,-59.7968,
5,Atuona,PF,76.26,light rain,-9.8000,-139.0333,
6,Road Town,VG,82.49,overcast clouds,18.4167,-64.6167,
7,Puerto Escondido,MX,76.17,overcast clouds,15.8500,-97.0667,
8,Hithadhoo,MV,82.02,scattered clouds,-0.6000,73.0833,
9,Airai,TL,81.54,scattered clouds,-8.9266,125.4092,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].isnull().sum()

0

In [13]:
# 8a. Create the output File (CSV)
output_data_file = "weather_data/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Max Temp} °F and {Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))